In [1]:
import numpy as np

import dolfinx

from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import fem, mesh, io, plot, log,default_scalar_type
from dolfinx.fem import (Constant, dirichletbc, Function, functionspace, Expression )
from dolfinx.fem.petsc import NonlinearProblem,LinearProblem
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.io.gmshio import model_to_mesh
import ufl
from ufl import (TestFunctions, TrialFunction, Identity, grad, det, div, dev, inv, tr, sqrt, conditional , gt, dx, inner, derivative, dot, ln, split,TestFunction,indices,as_tensor)
from basix.ufl import element, mixed_element, quadrature_element
from datetime import datetime
from dolfinx.plot import vtk_mesh

import pyvista
pyvista.set_jupyter_backend('client')
## Define temporal parameters

In [2]:
import gmsh

gmsh.initialize()
gmsh.open("meshes/thick_cylinder.geo")
gmsh.model.mesh.generate(2)
gmsh.model.mesh.optimize("Netgen")
domain, ct, ft = model_to_mesh(gmsh.model, MPI.COMM_WORLD, 0, gdim=2)
gmsh.finalize()

Info    : Reading 'meshes/thick_cylinder.geo'...
Info    : Done reading 'meshes/thick_cylinder.geo'
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Circle)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Circle)
Info    : Done meshing 1D (Wall 0.000245125s, CPU 0.000274s)
Info    : Meshing 2D...
Info    : Meshing surface 6 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.010169s, CPU 0.010146s)
Info    : 812 nodes 1625 elements
Info    : Optimizing mesh (Netgen)...
Info    : Done optimizing mesh (Wall 9.99993e-07s, CPU 2e-06s)


In [5]:
plotter = pyvista.Plotter()
vtkdata = vtk_mesh(domain, domain.topology.dim)
grid = pyvista.UnstructuredGrid(*vtkdata)
actor = plotter.add_mesh(grid, show_edges=True)
plotter.show()
plotter.close()

Widget(value="<iframe src='http://localhost:34801/index.html?ui=P_0xffff52ae2260_0&reconnect=auto' style='widt…

In [9]:
E = Constant(domain,70e3)
nu = Constant(domain,0.3)
lmbda = E * nu / (1 + nu) / (1 - 2 * nu)
mu = E / 2.0 / (1 + nu)
sig0 = Constant(domain,250.0)  # yield strength
Et = E / 100.0  # tangent modulus
H = E * Et / (E - Et)  # hardening modulus

Re, Ri = 1.3, 1.0  # external/internal radius


ds = ufl.Measure("ds", domain=domain, subdomain_data=ft)

In [10]:
deg_u = 2
deg_stress = 2

